In [2]:
!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt


--2021-01-14 07:38:02--  https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53933315 (51M) [text/plain]
Saving to: ‘glove.840B.300d.sst.txt.1’

glove.840B.300d.sst 100%[===================>]  51.43M   136MB/s    in 0.4s    

2021-01-14 07:38:02 (136 MB/s) - ‘glove.840B.300d.sst.txt.1’ saved [53933315/53933315]



In [3]:
!cp glove.840B.300d.sst.txt sample_data/glove.840B.300d.sst.txt

In [4]:
import pandas as pd
import numpy as np
import os
import pickle
import numpy as np
import re
import itertools
from collections import Counter
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import *
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import array
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing import sequence
from keras.regularizers import *
import tensorflow as tf
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
import timeit

Constructing training and test datasets

In [5]:
# Reading the csv file into a pandas dataframe object
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/NLP- Assignments/TWEET_DATA_IEEE_TCSS - Sheet1.csv', names=['Tweet', 'Tag'])


In [6]:
from sklearn.model_selection import train_test_split

# split data into train and test set in 80:20 ratio
train, test = train_test_split(dataset, test_size=0.2, random_state = 12)

In [7]:
# holds the test data tweets
test_data = test['Tweet'].tolist()

# holds the training data tweets
train_data = train['Tweet'].tolist()

# holds the test data labels
test_label = test['Tag'].tolist()

# holds the training data labels
train_label = train['Tag'].tolist()

In [8]:
# holds the entire tweet data
text_data = dataset['Tweet'].tolist()

# computing the maximum length of the tweets present
max1=0

for i in range(0,len(text_data)):
  max2=len(text_data[i])
  if max2 > max1 :
    max1 = max2

sequence_length = max1

In [9]:
# function to pad additional values so that all vectors are of equal length
def load_create_padded_data(X_train,maxlen=None):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(X_train)
  X_train = tokenizer.texts_to_sequences(X_train)
  X_train = pad_sequences(X_train,maxlen=maxlen)
  return X_train

In [10]:
# constructing padded vectors for the training and testing datasets
X_train = load_create_padded_data(X_train=train_data,maxlen=sequence_length)
X_test = load_create_padded_data(X_train=test_data,maxlen=sequence_length)

In [11]:
# constructing a tokenizer for thr trainig data
# word_index stores the distinct vocabulary of words that make up the tweets
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index

In [12]:
# function to create the embedding matrix
def load_create_embedding_matrix(word_index,vocab_size,emb_dim,emb_path):
  embedding_dict = {}

  f = open(emb_path,'rb')
  for line in f:
    fields = line.split()
    word = fields[0]
    w_e = np.asarray(fields[1:],dtype='float32')
    embedding_dict[word] = w_e
  f.close()

  allembs = np.stack(embedding_dict.values())
  emb_mean,emb_std = allembs.mean(),allembs.std()

  embedding_matrix = np.random.normal(emb_mean,emb_std,(vocab_size,emb_dim))

  for word,index in word_index.items():
    vector = embedding_dict.get(word)
    if vector is not None:
      embedding_matrix[index] = vector
  
  return embedding_matrix

In [13]:
# construction of the embedding matrix using the GLOVE embeddings
embedding_matrix = load_create_embedding_matrix(word_index,len(word_index)+1,300,'./glove.840B.300d.sst.txt')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
 #finding the unique set of labels used in the dataset for classification
label_dict = {}
index = 0

for label in train_label:
	if label not in label_dict:
		label_dict[label] = index
		index = index + 1

In [16]:
def create_label(label):
  Y = []
  for i in label:
    temp_array = np.zeros(len(label_dict))
    label_at_i = label_dict.get(i)
    temp_array[label_at_i] = 1
    Y.append(temp_array)
  return Y

In [17]:
Y_train = create_label(train_label)
Y_test = create_label(test_label)

y_train = np.array([i for i in Y_train])
y_test = np.array([i for i in Y_test])

embedding_dim = 300

**Deep Neural Network in Keras using 1-dimensional Convolutional Neural Network (CNN) (Filter size = 32,
kernel size= 3)**

In [18]:
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=len(word_index)+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=sequence_length)(inputs)
conv_0 = Conv1D(filters=32, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(embedding)
maxpool_0 = MaxPooling1D(pool_size=2)(conv_0)
dropout = Dropout(0.1)(maxpool_0)
conv_1 = Conv1D(filters=64, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(dropout)
maxpool_1 = MaxPooling1D(pool_size=2)(conv_1)
flatten = Flatten()(maxpool_1)
predictions = Dense(1000, activation='relu')(flatten)
predictions1 = Dense(len(label_dict), activation='softmax')(predictions)
adam = Adam(lr=0.01, decay=0.3)
model = Model(inputs=inputs, outputs=predictions1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))


predicted = model.predict(X_test)

Creating Model...
Epoch 1/30
94/94 [==============================] - 22s 220ms/step - loss: 11.9375 - accuracy: 0.4714 - val_loss: 1.4420 - val_accuracy: 0.5140
Epoch 2/30
94/94 [==============================] - 20s 217ms/step - loss: 1.3990 - accuracy: 0.5278 - val_loss: 1.4283 - val_accuracy: 0.5140
Epoch 3/30
94/94 [==============================] - 20s 216ms/step - loss: 1.3790 - accuracy: 0.5341 - val_loss: 1.4215 - val_accuracy: 0.5140
Epoch 4/30
94/94 [==============================] - 25s 268ms/step - loss: 1.3289 - accuracy: 0.5329 - val_loss: 1.4255 - val_accuracy: 0.5140
Epoch 5/30
94/94 [==============================] - 20s 218ms/step - loss: 1.2910 - accuracy: 0.5471 - val_loss: 1.4261 - val_accuracy: 0.5140
Epoch 6/30
94/94 [==============================] - 20s 216ms/step - loss: 1.2674 - accuracy: 0.5739 - val_loss: 1.4392 - val_accuracy: 0.5140
Epoch 7/30
94/94 [==============================] - 20s 217ms/step - loss: 1.2413 - accuracy: 0.6056 - val_loss: 1.4490 - v

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 310)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 310, 300)          4563600   
_________________________________________________________________
conv1d (Conv1D)              (None, 310, 32)           28832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 155, 32)           0         
_________________________________________________________________
dropout (Dropout)            (None, 155, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 155, 64)           6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 77, 64)            0     

In [20]:
Y_predicted=[]
for i in predicted:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
    
    
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print("\nAccuracy Score : \n\n", accuracy_score(Y_test, Y_predicted))
print("\nClassification Report : \n\n", classification_report(Y_test, Y_predicted,digits=4,))
print("\nConfusion Matrix : \n\n", confusion_matrix(Y_test, Y_predicted))


Accuracy Score : 

 0.502

Classification Report : 

               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        82
           1     0.5205    0.9533    0.6734       771
           2     0.2045    0.0516    0.0824       349
           3     0.0000    0.0000    0.0000       118
           4     0.0000    0.0000    0.0000        53
           5     0.0000    0.0000    0.0000        48
           6     0.0000    0.0000    0.0000        79

    accuracy                         0.5020      1500
   macro avg     0.1036    0.1436    0.1080      1500
weighted avg     0.3151    0.5020    0.3653      1500


Confusion Matrix : 

 [[  0  74   8   0   0   0   0]
 [  0 735  36   0   0   0   0]
 [  0 331  18   0   0   0   0]
 [  0 110   8   0   0   0   0]
 [  0  47   6   0   0   0   0]
 [  0  46   2   0   0   0   0]
 [  0  69  10   0   0   0   0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



**Deep Neural Network in Keras using Using Bi-directional Long Short Term Memory Network (Bi-LSTM) (72 units each)**

In [22]:
from tensorflow import keras
from tensorflow.keras import layers

# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(len(word_index)+1, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(72, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(72))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
Bi_LSTM_model = keras.Model(inputs, outputs)
Bi_LSTM_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         1947136   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 144)         115776    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 144)               124992    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 145       
Total params: 2,188,049
Trainable params: 2,188,049
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test))
predicted_new = model.predict(X_test)

Epoch 1/30
188/188 [==============================] - 25s 132ms/step - loss: 0.3302 - accuracy: 0.6400 - val_loss: 0.3479 - val_accuracy: 0.4787
Epoch 2/30
188/188 [==============================] - 25s 131ms/step - loss: 0.1925 - accuracy: 0.7333 - val_loss: 0.3936 - val_accuracy: 0.4380
Epoch 3/30
188/188 [==============================] - 25s 131ms/step - loss: 0.1397 - accuracy: 0.8006 - val_loss: 0.4645 - val_accuracy: 0.4667
Epoch 4/30
188/188 [==============================] - 25s 132ms/step - loss: 0.0898 - accuracy: 0.8867 - val_loss: 0.4890 - val_accuracy: 0.4147
Epoch 5/30
188/188 [==============================] - 24s 130ms/step - loss: 0.0618 - accuracy: 0.9273 - val_loss: 0.5346 - val_accuracy: 0.4107
Epoch 6/30
188/188 [==============================] - 25s 132ms/step - loss: 0.0444 - accuracy: 0.9532 - val_loss: 0.6272 - val_accuracy: 0.4200
Epoch 7/30
188/188 [==============================] - 25s 132ms/step - loss: 0.0337 - accuracy: 0.9643 - val_loss: 0.6074 - val_ac

In [24]:
Y_predicted_new = []
for i in predicted_new:
    pos=i.argmax()
    Y_predicted_new.append(pos)
    
    
    
Y_test_new = []
for i in y_test:
    pos = i.argmax()
    Y_test_new.append(pos)
    
    
    
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print("\nAccuracy Score : \n\n", accuracy_score(Y_test, Y_predicted_new))
print("\nClassification Report : \n\n", classification_report(Y_test, Y_predicted_new,digits=4,))
print("\nConfusion Matrix : \n\n", confusion_matrix(Y_test, Y_predicted_new))


Accuracy Score : 

 0.4026666666666667

Classification Report : 

               precision    recall  f1-score   support

           0     0.2143    0.0732    0.1091        82
           1     0.5204    0.6783    0.5890       771
           2     0.1658    0.0946    0.1204       349
           3     0.0842    0.0678    0.0751       118
           4     0.0000    0.0000    0.0000        53
           5     0.0000    0.0000    0.0000        48
           6     0.2446    0.4304    0.3119        79

    accuracy                         0.4027      1500
   macro avg     0.1756    0.1920    0.1722      1500
weighted avg     0.3373    0.4027    0.3591      1500


Confusion Matrix : 

 [[  6  40  18   5   0   6   7]
 [ 10 523 103  60   5  14  56]
 [  8 264  33  17   2   3  22]
 [  1  76  19   8   1   2  11]
 [  3  38   8   2   0   1   1]
 [  0  33   5   2   0   0   8]
 [  0  31  13   1   0   0  34]]


**Wrong matched Sentences- CNN model**

In [25]:
tag_numbers = {}
for key, value in label_dict.items():
  tag_numbers[value] = key

In [26]:
index_of_difference_new = []
for i in range(0, len(Y_predicted_new)):
  if Y_predicted_new[i] != Y_test_new[i]:
    index_of_difference_new.append((i, Y_test_new[i], Y_predicted_new[i]))

In [27]:
difference_dataframe_new = pd.DataFrame(columns={'Tweet', 'Actual Tag', 'Predicted Tag'})
for elt in index_of_difference_new:
  difference_dataframe_new = difference_dataframe_new.append({'Tweet' : dataset['Tweet'][elt[0]], 'Actual Tag' : tag_numbers[elt[1]], 'Predicted Tag' : tag_numbers[elt[2]]}, ignore_index=True)

In [29]:
difference_dataframe_new

,Tweet,Actual Tag,Predicted Tag
0,RT @stevesilberman: If you missed it: Sick bur...,QUE,STM
1,@jk_rowling & the Never-Ending Story: With a #...,STM,EXP
2,RT @somebadideas: The emotional & personal eff...,REQ,OTH
3,RT @jk_rowling: A 9-year-old Nigerian girl has...,EXP,OTH
4,RT @jk_rowling: I don't want to say too much m...,STM,OTH
...,...,...,...
891,Sweet is a comic book fan from way back...He's...,STM,EXP
892,@naomi0_0barton You mean Ursula Le Guin? She i...,SUG,EXP
893,Hard times are coming. We'll need writers who ...,STM,QUE
894,"@alsotheabyss Left Hand of Darkness too, but L...",SUG,STM
